In [ ]:
# Cell 1: Import libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import os

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline
pd.set_option('display.max_columns', None)

print("Libraries loaded!")
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Cell 2: Count total papers in dataset
print(" Counting total papers in dataset...")
print("(This takes 1-2 minutes for the full file)\n")

total_papers = 0
with open('/Users/aashigoyani/research-paper-recommender/rpr/data/raw/arxiv-metadata-oai-snapshot.json', 'r') as f:
    for i, line in enumerate(f):
        total_papers += 1
        
        # Progress update every 100k papers
        if (i + 1) % 100000 == 0:
            print(f"Counted {i + 1:,} papers...")

print(f"\nTotal papers in dataset: {total_papers:,}")

In [ ]:
# Cell 3: What categories do we have?
print(" Analyzing category distribution...")
print("(Loading first 50,000 papers to sample)\n")

category_counter = Counter()

with open('/Users/aashigoyani/research-paper-recommender/rpr/data/raw/arxiv-metadata-oai-snapshot.json', 'r') as f:
    for i, line in enumerate(f):
        if i >= 50000:  # Sample first 50K
            break
            
        paper = json.loads(line)
        categories = paper['categories'].split()
        category_counter.update(categories)
        
        if (i + 1) % 10000 == 0:
            print(f"Processed {i + 1:,} papers...")

# Show top categories
print("\n" + "=" * 60)
print("TOP 20 CATEGORIES (from 50K sample)")
print("=" * 60)

for cat, count in category_counter.most_common(20):
    percentage = (count / 50000) * 100
    print(f"{cat:20} : {count:6,} papers ({percentage:5.2f}%)")

# Identify CS categories
cs_categories = [cat for cat in category_counter.keys() if cat.startswith('cs.')]
print(f"\nFound {len(cs_categories)} Computer Science categories:")
print(cs_categories[:10])  # Show first 10

In [ ]:
# Cell 4: Extract Computer Science papers only

cs_papers = []
total_scanned = 0

with open('/Users/aashigoyani/research-paper-recommender/rpr/data/raw/arxiv-metadata-oai-snapshot.json', 'r') as f:
    for i, line in enumerate(f):
        paper = json.loads(line)
        total_scanned += 1
        
        # Check if any category starts with 'cs.'
        categories = paper['categories'].split()
        if any(cat.startswith('cs.') for cat in categories):
            cs_papers.append(paper)
        
        # Progress update
        if (i + 1) % 100000 == 0:
            print(f"Scanned {i + 1:,} papers | Found {len(cs_papers):,} CS papers")
        
        # Optional: Stop after finding 100K CS papers (remove this if you want all)
        if len(cs_papers) >= 100000:
            print(f"\n⏸  Stopping at 100,000 CS papers for manageable size")
            break

print(f"\n Total scanned: {total_scanned:,}")
print(f"CS papers found: {len(cs_papers):,}")
print(f"Percentage: {(len(cs_papers)/total_scanned)*100:.2f}%")

In [ ]:
# Cell 5: Convert to DataFrame
print("Converting to DataFrame...")

df_cs = pd.DataFrame(cs_papers)

print(f" DataFrame created!")
print(f"   Shape: {df_cs.shape}")
print(f"   Columns: {df_cs.columns.tolist()}\n")

# Show info
print("=" * 60)
print("DATASET INFO")
print("=" * 60)
df_cs.info()

print("\n" + "=" * 60)
print("SAMPLE RECORDS")
print("=" * 60)
df_cs.head(3)

In [ ]:
# Cell 6: Clean the data
print(" Cleaning data...\n")

# Store original size
original_size = len(df_cs)

# 1. Remove duplicates by paper ID
df_cs = df_cs.drop_duplicates(subset=['id'], keep='first')
print(f"1. Removed {original_size - len(df_cs)} duplicate papers")

# 2. Remove papers with very short abstracts (likely incomplete)
df_cs['abstract_length'] = df_cs['abstract'].str.len()
df_cs = df_cs[df_cs['abstract_length'] >= 100]
print(f"2. Removed papers with abstracts < 100 characters")

# 3. Remove papers with missing titles
df_cs = df_cs[df_cs['title'].notna()]
print(f"3. Removed papers with missing titles")

# 4. Extract year from update_date
df_cs['year'] = pd.to_datetime(df_cs['update_date']).dt.year
print(f"4. Extracted year from dates")

# 5. Clean text fields (remove extra whitespace, LaTeX commands)
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove LaTeX commands
    text = re.sub(r'\$.*?\$', '', text)  # Remove inline math
    text = re.sub(r'\\[a-zA-Z]+\{.*?\}', '', text)  # Remove LaTeX commands
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df_cs['title_clean'] = df_cs['title'].apply(clean_text)
df_cs['abstract_clean'] = df_cs['abstract'].apply(clean_text)
print(f"5. Cleaned text fields")

print(f"\n Final dataset size: {len(df_cs):,} papers")
print(f"   Removed {original_size - len(df_cs):,} papers total")

In [ ]:
# Cell 7: Analyze the cleaned dataset
print("=" * 60)
print("CLEANED DATASET STATISTICS")
print("=" * 60)

# Year distribution
print("\n Papers by Year (Last 10 years):")
year_dist = df_cs['year'].value_counts().sort_index().tail(10)
for year, count in year_dist.items():
    print(f"   {year}: {count:,} papers")

# Category distribution
print("\n  Top CS Sub-categories:")
cs_subcats = Counter()
for cats in df_cs['categories']:
    cs_cats = [c for c in cats.split() if c.startswith('cs.')]
    cs_subcats.update(cs_cats)

for cat, count in cs_subcats.most_common(10):
    percentage = (count / len(df_cs)) * 100
    print(f"   {cat:15} : {count:6,} papers ({percentage:5.2f}%)")

# Abstract length
print(f"\n📏 Abstract Length Statistics:")
print(f"   Mean: {df_cs['abstract_length'].mean():.0f} characters")
print(f"   Median: {df_cs['abstract_length'].median():.0f} characters")
print(f"   Min: {df_cs['abstract_length'].min():.0f} characters")
print(f"   Max: {df_cs['abstract_length'].max():.0f} characters")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Year distribution
year_dist.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Papers by Year', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Number of Papers')

# Abstract length distribution
axes[1].hist(df_cs['abstract_length'], bins=50, color='coral', edgecolor='black')
axes[1].set_title('Abstract Length Distribution', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Length (characters)')
axes[1].set_ylabel('Frequency')
axes[1].axvline(df_cs['abstract_length'].mean(), color='red', 
                linestyle='--', label='Mean')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Cell 8: Create manageable sample for development
print(" Creating sample datasets...\n")

# Create different sized samples
sample_sizes = {
    'small': 10000,
    'medium': 50000,
    'large': 100000
}

samples = {}
for name, size in sample_sizes.items():
    if len(df_cs) >= size:
        # Random sample
        sample = df_cs.sample(n=size, random_state=42)
        samples[name] = sample
        print(f" Created '{name}' sample: {size:,} papers")
    else:
        print(f"  Dataset smaller than {size:,}, using all {len(df_cs):,} papers")
        samples[name] = df_cs

# We'll work with the small sample (10K) for development
df_sample = samples['small']

print(f"\n Working with: {len(df_sample):,} papers")

In [ ]:
# Cell 9: Save processed datasets
import os

# Create processed data directory if it doesn't exist
os.makedirs('data/processed', exist_ok=True)

print(" Saving processed data...\n")

# Save the sample (for quick development)
df_sample.to_json('data/processed/papers_sample_10k.json', 
                  orient='records', 
                  lines=True)
print(f"Saved: data/processed/papers_sample_10k.json ({len(df_sample):,} papers)")

# Save medium sample if exists
if 'medium' in samples and len(samples['medium']) > len(df_sample):
    samples['medium'].to_json('data/processed/papers_sample_50k.json',
                              orient='records',
                              lines=True)
    print(f"Saved: data/processed/papers_sample_50k.json ({len(samples['medium']):,} papers)")

# Save full CS dataset
df_cs.to_json('data/processed/papers_cs_full.json',
              orient='records',
              lines=True)
print(f"Saved: data/processed/papers_cs_full.json ({len(df_cs):,} papers)")

# Save a CSV version of the sample for easy viewing
df_sample[['id', 'title_clean', 'authors', 'year', 'categories']].to_csv(
    'data/processed/papers_sample_10k_preview.csv',
    index=False
)
print(f"Saved: data/processed/papers_sample_10k_preview.csv (for preview)")

print(f"\n All data saved successfully!")

In [ ]:
# Cell 10: Final summary
print("=" * 60)
print(" DATA PREPROCESSING COMPLETE!")
print("=" * 60)

print(f"\n Processed Files Created:")
print(f"   1. papers_sample_10k.json         - {len(df_sample):,} papers (for development)")
print(f"   2. papers_cs_full.json            - {len(df_cs):,} papers (full CS dataset)")
print(f"   3. papers_sample_10k_preview.csv  - Preview in spreadsheet format")

print(f"\n Dataset Statistics:")
print(f"   Total papers scanned: {total_scanned:,}")
print(f"   CS papers extracted: {len(df_cs):,}")
print(f"   Year range: {df_cs['year'].min()} - {df_cs['year'].max()}")
print(f"   Unique categories: {len(cs_subcats)}")
